Необходимые модули:

In [17]:
import numpy as np
import copy

Входные данные:

In [18]:
test_cases = [
    [
        syst := [
            [-1, 1, 1, 0, 0],
            [1, 0, 0, 1, 0],
            [0, 1, 0, 0, 1],
        ],
        b := [1, 3, 2],
        c := [1, 1, 0, 0, 0],
        x := [0, 0, 1, 3, 2],
        Jb := [2, 3, 4],
        res := [3, 2, 2, 0, 0],
    ],
    [
        syst := [
            [1, 2, 1, 0, 0, 0],
            [2, 1, 0, 1, 0, 0],
            [1, 0, 0, 0, 1, 0],
            [0, 1, 0, 0, 0, 1]
        ],
        b := [10, 11, 5, 4],
        c := [20, 26, 0, 0, 0, 1],
        x := [0, 0, 10, 11, 5, 4],
        Jb := [2, 3, 4, 5],
        res := [4, 3, 0, 0, 1, 1],
    ],
    [
        syst := [
            [1, 1, 1, 0, 0, 0],
            [2, 5, 0, 1, 0, 0],
            [1, 0, 0, 0, 1, 0],
            [1, 0, 0, 0, 0, 1]
        ],
        b := [9, 30, 5, 7],
        c := [19, 29, -1, 0, 0, 0],
        x := [0, 0, 9, 30, 5, 7],
        Jb := [2, 3, 4, 5],
        res := [5, 4, 0, 0, 0, 2],
    ]
]

Функции, которые понадобятся для решения:

In [19]:
def InvMatrix(A, invA, x, i):
    A[:,i] = x[:]
    l = invA.dot(x)
    if (l[i] == 0):
        print("Матрица A необратима")
        return
    ln = l.copy()
    ln[i] = -1
    ls = (-1/l[i]) * ln
    Q = np.eye(len(A))
    Q[:,i] = ls
    return np.matmul(Q, invA)

def Simplex(syst, b, c, x, Jb):
    syst = np.array(copy.deepcopy(syst))
    b = np.array(b[:])
    c = np.array(c[:])
    x = np.array(x[:])
    Jb = np.array(Jb[:])

    n = len(x)

    j0_new = -1

    # По множеству базисных индексов составляются матрица Ab, вектор cb. 
    cb = []
    Ab = np.zeros([len(Jb)] * 2)
    for i, j in enumerate(Jb):
        Ab[:, i] = syst[:, j]
        cb.append(c[j])

    invAb = np.linalg.inv(Ab)

    while True:
        # вектор потенциалов:
        u = np.dot(cb, invAb)

        # вектор оценок:
        delta = np.matmul(u, syst) - c

        optimal = True
        for i in range(n):
            if i not in Jb and delta[i] < 0:
                optimal = False
                break
        
        if optimal:
            F = np.dot(c, x)
            print("План отпимальный")
            print(f"F{tuple(x)} = {F}")
            
            return x, F, Jb

        j0 = 0
        for i in range(len(delta)):
            if (delta[i] < 0 and i not in Jb):
                j0 = i
                break

        z = np.matmul(invAb, syst[:,j0])
    
        thetas = []  
        for i in range(len(Jb)):
            thetas.append(x[Jb[i]] / z[i] if z[i] > 0 else np.inf)

        theta0 = min(thetas)
        if (theta0 == np.inf):
            print('Целевая функция неограничена сверху')
            return
        j0_new = thetas.index(theta0)

        # если один из индексов базисный, а другой нет -
        # то меняем их местами
        Jb_new = Jb.copy()
        Jb_new[j0_new] = j0

        # обновляем компоненты текущего базисного множества
        for i in range(len(x)):
            if (i not in Jb):
                x[i] = 0

        for i, j in enumerate(Jb):
            x[j] -= theta0 * z[i]
        x[j0] = theta0

        # базис для следующих итераций
        Jb[:] = Jb_new

        # По множеству базисных индексов составляются матрица Ab, вектор cb. 
        cb = [] 
        for j in Jb:
            cb.append(c[j])

        # находим матрицу, обратную Ab,
        # с заменой столбца
        invAb = InvMatrix(Ab, invAb, syst[:,j0], j0_new)

Решение:

In [20]:
for test_case in test_cases:
    syst, b, c, x, Jb, res = test_case
    x, F, Jb = Simplex(syst, b, c, x, Jb)
    print(f'правильный ответ:\nF{tuple(res)} = {np.array(c).dot(res)}\n\n')

delta = array([-1., -1.,  0.,  0.,  0.])
delta = array([ 0., -1.,  0.,  1.,  0.])
delta = array([0., 0., 0., 1., 1.])
План отпимальный
F(3, 2, 2, 0, 0) = 5
правильный ответ:
F(3, 2, 2, 0, 0) = 5


delta = array([-20., -25.,   0.,   0.,   0.,   0.])
delta = array([  0., -25.,   0.,   0.,  20.,   0.])
delta = array([  0.,   0.,   0.,  25., -30.,   0.])
delta = array([ 0.,  0., 10.,  5.,  0.,  0.])
План отпимальный
F(4, 3, 0, 0, 1, 1) = 159
правильный ответ:
F(4, 3, 0, 0, 1, 1) = 159


delta = array([-20., -30.,   0.,   0.,   0.,   0.])
delta = array([  0., -30.,   0.,   0.,  20.,   0.])
delta = array([  0.,   0.,  30.,   0., -10.,   0.])
delta = array([ 0.00000000e+00,  0.00000000e+00,  1.33333333e+01,  3.33333333e+00,
       -1.60982339e-15,  0.00000000e+00])
План отпимальный
F(5, 4, 0, 0, 0, 2) = 211
правильный ответ:
F(5, 4, 0, 0, 0, 2) = 211


